# Maximal Clique Problem

## Graph Generation

It was provided the functions below for graph generation and the solution to solve the Maximal Clique Problem:

In [1]:
import networkx as nx
import random

def create_graph(num_vertices, probabilidade_conexao, nome_arquivo):

    # Crie um grafo aleatório densamente conectado
    grafo = nx.fast_gnp_random_graph(num_vertices, probabilidade_conexao)

    # Abra o arquivo para escrita
    with open(nome_arquivo, 'w') as arquivo:
        # Escreva a quantidade de vértices e número de arestas na primeira linha
        arquivo.write(f"{num_vertices} {grafo.number_of_edges()}\n")

        # Escreva as arestas no formato de lista de adjacência
        for aresta in grafo.edges():
            arquivo.write(f"{aresta[0]+1} {aresta[1]+1}\n")  # +1 para ajustar os índices (começando em 1)

    print(f"Grafo densamente conectado gerado e salvo em '{nome_arquivo}'.")

In [15]:
def max_clique(nome_arquivo,grafo):
    # Abrir o arquivo e pular a primeira linha
    with open(nome_arquivo, 'r') as arquivo:
        next(arquivo)  # Pula a primeira linha

        # Lê o grafo a partir das linhas restantes
        G = nx.parse_adjlist(arquivo)

    # Encontrar todas as cliques maximais
    cliques_maximais = list(nx.find_cliques(G))

    # Encontrar a clique máxima (a maior)
    clique_maxima = max(cliques_maximais, key=len)

    print(f"Clique máxima encontrada grafo {grafo} vértices:", clique_maxima)

In [19]:
lista_vertices = [10,15,20,30, 40, 50]

for i in lista_vertices:
    create_graph(i, 0.7, f"grafos/grafo_{i}_vertices.txt")

Grafo densamente conectado gerado e salvo em 'grafos/grafo_10_vertices.txt'.
Grafo densamente conectado gerado e salvo em 'grafos/grafo_15_vertices.txt'.
Grafo densamente conectado gerado e salvo em 'grafos/grafo_20_vertices.txt'.
Grafo densamente conectado gerado e salvo em 'grafos/grafo_30_vertices.txt'.
Grafo densamente conectado gerado e salvo em 'grafos/grafo_40_vertices.txt'.
Grafo densamente conectado gerado e salvo em 'grafos/grafo_50_vertices.txt'.


## Algorithm Overview

## Validation

### Resultado Python

In [17]:
for i in lista_vertices:
    max_clique(f"grafos/grafo_{i}_vertices.txt",i)

Clique máxima encontrada grafo 10 vértices: ['5', '8', '6', '4', '1']
Clique máxima encontrada grafo 15 vértices: ['4', '5', '10', '3', '1']
Clique máxima encontrada grafo 20 vértices: ['7', '20', '6', '8', '10', '18', '13']
Clique máxima encontrada grafo 30 vértices: ['20', '4', '27', '6', '8', '11', '5', '9', '10']
Clique máxima encontrada grafo 40 vértices: ['9', '14', '31', '39', '34', '16', '20', '37', '36', '28']
Clique máxima encontrada grafo 50 vértices: ['50', '44', '21', '49', '2', '41', '28', '24', '33', '32', '14']


## Results

## Conclusion